In [8]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

try:
    import policyengine_uk
    print("PolicyEngine imported OK")
except ModuleNotFoundError as e:
    print("Import error:", e)



Python executable: /Users/Nil/policyengine-311/bin/python3.11
Python version: 3.11.14 (main, Oct  9 2025, 16:16:55) [Clang 17.0.0 (clang-1700.0.13.3)]
PolicyEngine imported OK


## run PolicyEngine

In [32]:
import policyengine_core.data as coredata
import os

print(os.listdir(os.path.dirname(coredata.__file__)))


['__init__.py', '__pycache__', 'dataset.py']


In [33]:
import policyengine_core
dir(policyengine_core)


['Microsimulation',
 'Simulation',
 'TaxBenefitSystem',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'commons',
 'data',
 'data_storage',
 'data_structures',
 'entities',
 'enums',
 'errors',
 'experimental',
 'holders',
 'model_api',
 'parameters',
 'periods',
 'populations',
 'projectors',
 'reforms',
 'simulations',
 'taxbenefitsystems',
 'taxscales',
 'tools',
 'tracers',
 'types',
 'variables',
 'warnings']

In [34]:
import os

for root, dirs, files in os.walk("/Users/Nil/policyengine-311/lib", topdown=True):
    for f in files:
        if f.endswith(".h5"):
            print(os.path.join(root, f))


/Users/Nil/policyengine-311/lib/python3.11/site-packages/h5py/tests/data_files/vlen_string_s390x.h5
/Users/Nil/policyengine-311/lib/python3.11/site-packages/h5py/tests/data_files/vlen_string_dset_utc.h5
/Users/Nil/policyengine-311/lib/python3.11/site-packages/h5py/tests/data_files/vlen_string_dset.h5


In [35]:
for root, dirs, files in os.walk("/Users/Nil/policyengine-311/lib", topdown=True):
    for f in files:
        if "example" in f.lower() or "synthetic" in f.lower():
            print(os.path.join(root, f))


/Users/Nil/policyengine-311/lib/python3.11/site-packages/IPython/testing/plugin/dtexample.py
/Users/Nil/policyengine-311/lib/python3.11/site-packages/IPython/testing/plugin/test_exampleip.txt
/Users/Nil/policyengine-311/lib/python3.11/site-packages/IPython/testing/plugin/test_example.txt
/Users/Nil/policyengine-311/lib/python3.11/site-packages/IPython/testing/plugin/__pycache__/dtexample.cpython-311.pyc
/Users/Nil/policyengine-311/lib/python3.11/site-packages/jedi/third_party/typeshed/third_party/3/docutils/examples.pyi
/Users/Nil/policyengine-311/lib/python3.11/site-packages/numpy/typing/tests/data/reveal/nbit_base_example.pyi
/Users/Nil/policyengine-311/lib/python3.11/site-packages/setuptools/tests/config/setupcfg_examples.txt


## test the PolicyEngine model

In [44]:
import pandas as pd

# ---- PERSON TABLE ----
import pandas as pd

# ---- PERSON TABLE ----
person = pd.DataFrame({
    "person_id":      [1, 2, 3, 4],
    "benunit_id":     [1, 1, 2, 3],
    "household_id":   [1, 1, 2, 3],
    "age":            [45, 43, 30, 10],
    "sex":            [1, 0, 1, 0],
    "employment_income": [30000, 0, 15000, 0],
    "pension_income":    [0, 12000, 0, 0],
})

person["is_employed"] = (person["employment_income"] > 0).astype(int)
person["disability"]  = 0
person["student"]     = (person["age"] < 18).astype(int)

# 🔁 IMPORTANT: rename to what PolicyEngine expects
person = person.rename(columns={
    "benunit_id":   "person_benunit_id",
    "household_id": "person_household_id",
})

person.head()


,person_id,person_benunit_id,person_household_id,age,sex,employment_income,pension_income,is_employed,disability,student
0,1,1,1,45,1,30000,0,1,0,0
1,2,1,1,43,0,0,12000,0,0,0
2,3,2,2,30,1,15000,0,1,0,0
3,4,3,3,10,0,0,0,0,0,1


In [47]:
# ---- BENUNIT TABLE ----
# ---- BENUNIT TABLE ----
benunit = (
    person.groupby("person_benunit_id")
    .agg(
        household_id=("person_household_id", "first"),
        adults=("age", lambda s: (s >= 18).sum()),
        children=("age", lambda s: (s < 18).sum()),
    )
    .reset_index()
)

# rename back so the column is literally called benunit_id
benunit = benunit.rename(columns={"person_benunit_id": "benunit_id"})

benunit["rent"]            = 0
benunit["council_tax"]     = 0
benunit["childcare_costs"] = 0

benunit.head()



,benunit_id,household_id,adults,children,rent,council_tax,childcare_costs
0,1,1,2,0,0,0,0
1,2,2,1,0,0,0,0
2,3,3,0,1,0,0,0


In [48]:
# ---- HOUSEHOLD TABLE ----
household = (
    person.groupby("person_household_id")
    .agg(
        num_benefit_units=("person_benunit_id", "nunique"),
    )
    .reset_index()
)

household = household.rename(columns={"person_household_id": "household_id"})

household["region"]               = 1
household["tenure"]               = 1
household["tenure_type"]          = "RENT_PRIVATELY"
household["gross_rent"]           = 0
household["rent"]                 = 0
household["council_tax"]          = 0
household["local_authority_code"] = 0

household.head()




,household_id,num_benefit_units,region,tenure,tenure_type,gross_rent,rent,council_tax,local_authority_code
0,1,1,1,1,RENT_PRIVATELY,0,0,0,0
1,2,1,1,1,RENT_PRIVATELY,0,0,0,0
2,3,1,1,1,RENT_PRIVATELY,0,0,0,0


In [49]:
from policyengine_uk.data.dataset_schema import UKSingleYearDataset
from policyengine_uk import Microsimulation

dataset = UKSingleYearDataset(
    person=person,
    benunit=benunit,
    household=household,
    fiscal_year=2025,
)

sim = Microsimulation(dataset=dataset)

results = sim.calculate_dataframe(
    ["household_id", "income_tax", "household_net_income"],
    period=2025,
)

results.head()



,household_id,income_tax,household_net_income
0,1,3234.0,NaN
1,2,486.0,NaN
2,3,0.0,NaN
